In [101]:
import pandas as pd
file = 'I-SPY_1_All_Patient_Clinical_and_Outcome_Data.xlsx'

# load and set index
ISPY = pd.read_excel(file, sheetname='predictors')
ISPY = ISPY.set_index('SUBJECTID')

# drop NaN
ISPY.dropna(inplace=True)
# drop Columns I don't need
ISPY.drop(['DataExtractDt','Her2MostPos','HR_HER2_CATEGORY','HR_HER2_STATUS'], axis=1,inplace=True)

#encode race and drop initial variable
ISPY = ISPY.join(pd.get_dummies(ISPY['race_id'], prefix=['Race']))
ISPY.drop(['race_id'], axis=1,inplace=True)

In [102]:
ISPY.head(1)

,age,ERpos,PgRpos,HR Pos,BilateralCa,Laterality,MRI LD Baseline,MRI LD 1-3dAC,MRI LD InterReg,MRI LD PreSurg,['Race']_0,['Race']_1,['Race']_3,['Race']_4,['Race']_5,['Race']_50
SUBJECTID,,,,,,,,,,,,,,,,
1001,38.73,1,0,1,0,1,88,78,30,14,0,1,0,0,0,0


In [103]:
# load and set index of predictors
outcomes_df = pd.read_excel(file, sheetname='outcomes')
outcomes_df = outcomes_df.set_index('SUBJECTID')

In [106]:
#merge PCR and predictors
df = ISPY.join(outcomes_df['PCR'])


,age,ERpos,PgRpos,HR Pos,BilateralCa,Laterality,MRI LD Baseline,MRI LD 1-3dAC,MRI LD InterReg,MRI LD PreSurg,['Race']_0,['Race']_1,['Race']_3,['Race']_4,['Race']_5,['Race']_50,PCR
SUBJECTID,,,,,,,,,,,,,,,,,
1001,38.73,1,0,1,0,1,88,78,30,14,0,1,0,0,0,0,0
1002,37.79,1,1,1,0,2,29,26,66,16,0,1,0,0,0,0,0
1003,49.83,1,1,1,0,1,50,64,54,46,0,1,0,0,0,0,0
1004,48.28,0,0,0,0,1,91,90,99,43,0,1,0,0,0,0,0
1005,45.80,1,1,1,0,1,98,109,60,42,0,1,0,0,0,0,0
1008,64.51,1,1,1,1,2,45,49,47,32,0,1,0,0,0,0,0
1009,40.66,0,0,0,0,1,75,66,57,7,0,0,0,1,0,0,0
1010,41.53,0,0,0,0,1,60,52,20,3,0,1,0,0,0,0,1
1011,40.82,0,0,0,0,2,68,68,56,47,0,0,0,1,0,0,1
